# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [5]:
sales = graphlab.SFrame('kc_house_data.gl 2/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [6]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before creating a new feature.
sales['floors'] = sales['floors'].astype(int) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [7]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [8]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.496456     | 6981926.539042     | 427729.501760 |
PROGRESS: | 2         | 3        | 0.000002  | 1.556323     | 6861970.573528     | 393259.158552 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [14]:
list(model_all.get('coefficients'))

[{'index': None, 'name': '(intercept)', 'value': 274952.6204401374},
 {'index': None, 'name': 'bedrooms', 'value': 0.0},
 {'index': None, 'name': 'bedrooms_square', 'value': 0.0},
 {'index': None, 'name': 'bathrooms', 'value': 8483.951487981589},
 {'index': None, 'name': 'sqft_living', 'value': 24.423802255110022},
 {'index': None, 'name': 'sqft_living_sqrt', 'value': 351.09783334319667},
 {'index': None, 'name': 'sqft_lot', 'value': 0.0},
 {'index': None, 'name': 'sqft_lot_sqrt', 'value': 0.0},
 {'index': None, 'name': 'floors', 'value': 0.0},
 {'index': None, 'name': 'floors_square', 'value': 0.0},
 {'index': None, 'name': 'waterfront', 'value': 0.0},
 {'index': None, 'name': 'view', 'value': 0.0},
 {'index': None, 'name': 'condition', 'value': 0.0},
 {'index': None, 'name': 'grade', 'value': 850.4273639766192},
 {'index': None, 'name': 'sqft_above', 'value': 20.07776545164483},
 {'index': None, 'name': 'sqft_basement', 'value': 0.0},
 {'index': None, 'name': 'yr_built', 'value': 0.0

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [15]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [25]:

import numpy as np
rss_list = []
for penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=penalty)
    print sum((model.predict(validation) - validation['price']) ** 2), penalty


6.28412158085e+14 10.0
6.28412158283e+14 31.6227766017
6.28412158907e+14 100.0
6.28412160882e+14 316.227766017
6.28412167128e+14 1000.0
6.2841218688e+14 3162.27766017
6.28412249343e+14 10000.0
6.28412446908e+14 31622.7766017
6.2841307204e+14 100000.0
6.28415052663e+14 316227.766017
6.2842135376e+14 1000000.0
6.28441657748e+14 3162277.66017
6.28509646259e+14 10000000.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [33]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=10.0 )
for item in list(model.get('coefficients')):
    print item

{'index': None, 'name': '(intercept)', 'value': 19772.58267172916}
{'index': None, 'name': 'bedrooms', 'value': 8163.519951991132}
{'index': None, 'name': 'bedrooms_square', 'value': 963.2862071693976}
{'index': None, 'name': 'bathrooms', 'value': 25578.429778713107}
{'index': None, 'name': 'sqft_living', 'value': 39.34515228908316}
{'index': None, 'name': 'sqft_living_sqrt', 'value': 1139.3138470621443}
{'index': None, 'name': 'sqft_lot', 'value': 0.005967749687844386}
{'index': None, 'name': 'sqft_lot_sqrt', 'value': 151.2053544260693}
{'index': None, 'name': 'floors', 'value': 20205.47120028384}
{'index': None, 'name': 'floors_square', 'value': 11652.108169112526}
{'index': None, 'name': 'waterfront', 'value': 602998.6843476704}
{'index': None, 'name': 'view', 'value': 93445.23381910368}
{'index': None, 'name': 'condition', 'value': 6883.581066001603}
{'index': None, 'name': 'grade', 'value': 6290.221869170764}
{'index': None, 'name': 'sqft_above', 'value': 43.61984924330976}
{'inde

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [34]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [35]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [44]:
non_zeros = []
for penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=penalty)
    #print len(filter(lambda x : x != 0.0, model['coefficients']['value'])), penalty
    print model['coefficients']['value'].nnz(), penalty
    


18 100000000.0
18 127427498.57
18 162377673.919
18 206913808.111
17 263665089.873
17 335981828.628
17 428133239.872
17 545559478.117
17 695192796.178
16 885866790.41
15 1128837891.68
15 1438449888.29
13 1832980710.83
11 2335721469.09
10 2976351441.63
6 3792690190.73
5 4832930238.57
3 6158482110.66
1 7847599703.51
1 10000000000.0


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [45]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [46]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [51]:
rss_list = []
for penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=penalty)
    rss = sum((model.predict(validation) - validation['price']) ** 2)
    print rss, penalty, model['coefficients']['value'].nnz()

9.64876826273e+14 2976351441.63 10
9.72840378296e+14 3019316638.95 10
9.81196599802e+14 3062281836.27 10
9.89653612279e+14 3105247033.59 10
9.98361723081e+14 3148212230.91 10
1.00822970436e+15 3191177428.24 9
1.01702550003e+15 3234142625.56 9
1.02492450555e+15 3277107822.88 8
1.0299794723e+15 3320073020.2 7
1.03293901937e+15 3363038217.52 7
1.03646285326e+15 3406003414.84 7
1.0401993584e+15 3448968612.16 7
1.04511992951e+15 3491933809.48 7
1.05027158412e+15 3534899006.8 7
1.05541689503e+15 3577864204.12 7
1.06058634457e+15 3620829401.45 7
1.06475371829e+15 3663794598.77 6
1.06876205082e+15 3706759796.09 6
1.07276526773e+15 3749724993.41 6
1.07695101957e+15 3792690190.73 6


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [55]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=3320073020.2)

for item in list(model['coefficients']):
    print item

{'index': None, 'name': '(intercept)', 'value': 215675.92441733723}
{'index': None, 'name': 'bedrooms', 'value': 1228.595822930348}
{'index': None, 'name': 'bedrooms_square', 'value': 0.0}
{'index': None, 'name': 'bathrooms', 'value': 16669.44494327836}
{'index': None, 'name': 'sqft_living', 'value': 33.089101148518495}
{'index': None, 'name': 'sqft_living_sqrt', 'value': 728.7447126695339}
{'index': None, 'name': 'sqft_lot', 'value': 0.0}
{'index': None, 'name': 'sqft_lot_sqrt', 'value': 0.0}
{'index': None, 'name': 'floors', 'value': 0.0}
{'index': None, 'name': 'floors_square', 'value': 0.0}
{'index': None, 'name': 'waterfront', 'value': 0.0}
{'index': None, 'name': 'view', 'value': 0.0}
{'index': None, 'name': 'condition', 'value': 0.0}
{'index': None, 'name': 'grade', 'value': 3155.8153837610166}
{'index': None, 'name': 'sqft_above', 'value': 31.080485027042005}
{'index': None, 'name': 'sqft_basement', 'value': 0.0}
{'index': None, 'name': 'yr_built', 'value': 0.0}
{'index': None,